## Номер документа

Номер документа часто указывается после символа No или N и может состоять из цифр, букв и спецсимволов. Примеры номеров:

* 67-ФЗ,

* 324-01-ЗМО,

* 824н.

Номер документа необходимо извлечь из текста «как есть», не подвергая дополнительной обработке.
Поле number должно содержать извлеченную строку. Чтобы скомпенсировать влияние ошибок распознавания символов в исходных текстах, при сравнении предсказанного и настоящего номера проверяющей системой регистр букв не учитывается.


In [2]:
import re
import json

In [106]:
reg_exprs = [
    re.compile(r' № [\w\-/А-Яа-я]+ *'),
    re.compile(r'\n№ [\w\-/А-Яа-я]+\n*'),
    re.compile(r'№[\w\-/А-Яа-я]+')
]

In [100]:
with open('train/txts/0a5a5bf22a018bd5daf50c44b2e6a68f4bb54ef5.txt', 'r') as f:
    doc = f.read()
for expr in reg_exprs:
    print(expr.findall(doc))

[' № 104-па ']
[]
['№104-па']


In [69]:
names, y = [], []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        names.append(doc_dict['id'])
        y.append(doc_dict['label']['number'])

In [70]:
for doc_name in names:
    with open('train/txts/' + doc_name + '.txt', 'r') as f:
        doc = f.read()
        res = []
        for expr in reg_exprs:
            res += expr.findall(doc)
        if not res:
            print(doc)

ПОСТАНОВЛЕНИК

02 ноября 2015 г. л 23/557-П

Экз.№

 

г Ульяновск

О внесении изменений в государственную программу
Ульяновской области "Развитие государственной ветеринарной
службы Ульяновской области в 2014-2018 годах"

Правительство Ульяновской области п остановляет:

1. Утвердить прилагаемые изменения в государственную программу
Ульяновской области "Развитие государственной ветеринарной службы
Ульяновской области в 2014-2018 годах", утверждённую постановлением
Правительства - Ульяновской - области от - 11.09.2013  №  37/421-П
"Об утверждении государственной программы Ульяновской области "Развитие
государственной  ветеринарной службы Ульяновской области в 20142018 годах".

2. Настоящее постановление вступает в силу с 01 января 2016 года.

  
 

Губернатор - Председа б

Правительства области { 3 С.И.МорозовИтого по подпрограмме Департамент Всего 5688288
в том числе:

13

      
    
      
 

     
    
     
      
     
     
  

      

  
  
  

ветеринарии
бюджетные 15688288
ас

In [107]:
def extract_number(doc):
    number = ''
    matches = []
    for expr in reg_exprs:
        matches += expr.findall(doc)
    if matches:
        return matches[0].strip()[1:].strip()  # убираем номер
    return number

In [108]:
def predict(test):
    results = []
    for doc in test:
        number = extract_number(doc)
        prediction = {"type": "",
                      "date": "",
                      "number": number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [3]:
train = []
labeled = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

In [110]:
from eval_module import quality

predicted = predict(train)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.7520681875,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [111]:
predicted[:20]

[{'type': '', 'date': '', 'number': '35-ФЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '52-ФЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '28', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '114-ФЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '38-ФЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '48', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '1-ФКЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '12', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '17', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '52', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '27-ФЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '50', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '300-ФЗ', 'authority': '', 'name': ''},
 {'type': '', 'date': '', 'number': '1', 

Для каждого типа документа свой извлекатор:

* указ - в конце документа на отдельной строке № 807, может быть и в начале № 256-уг

* постановление, распоряжение, приказ - в начале после даты и места (или перед местом) в конце строки № 688-п, №_70-пп_, 63-пр,  № 25/2-г

* закон, федеральный закон - в конце документа № 89-ОЗ

Особые случаи:

* № _ 643-уГ

* 20 ноября 2014 года

№ 596-5-ЗКО

в постановлении в конце документа

* 02 ноября 2015 г. л 23/557-П в постановлении в начале (будем выбирать в качестве номера окончание строки)

In [4]:
date_reg_exprs = [
    re.compile(r'от \d?\d [А-Яа-я]+ \d\d\d\d[^\n]+'),
    re.compile(r'\d\d\.\d\d\.\d\d\d\d[^\n]+'),
    re.compile(r'\n\d?\d [А-Яа-я]+ \d\d\d\d[^\n]+')
]

number_reg_exprs = [
    re.compile(r' *№[_ ]+([\w\-/А-Яа-я]+) *'),
    re.compile(r'\n№[_ ]+([\w\-/А-Яа-я]+)\n*'),
    re.compile(r'№([\w\-/А-Яа-я]+)')
]

In [23]:
s = "№ 876 \n № 897"
list(map(lambda x: x.group(1), list(number_reg_exprs[0].finditer(s))))

['876', '897']

In [28]:
number_reg_exprs[0].search(s).group()

'№ 876 '

In [32]:
from typing import List

class NumberExtractor: 
    def predict(self, test: List[str], types: List[str]) -> List[str]:
        pred_numbers = []
        for doc, doc_type in zip(test, types):
            matches = []
            for expr in number_reg_exprs:
                matches += list(map(lambda x: x.group(1), list(expr.finditer(doc))))
            if matches:
                if doc_type in ['постановление', 'распоряжение', 'приказ']:
                    pred_numbers.append(matches[0])
                else:
                    pred_numbers.append(matches[-1])
            else:
                for expr in date_reg_exprs:
                    match = expr.search(doc)
                    if match:
                        matches = match.group()
                        break
                if matches:
                    pred_numbers.append(matches.strip().split(' ')[-1])
                else:
                    pred_numbers.append('')
        return pred_numbers

In [34]:
from eval_module import quality
from type_extractor import TypeExtractor
from sklearn.model_selection import train_test_split

train = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        with open(docname, 'r') as f:
            train.append((f.read(), doc_dict['label']))
train_data, test = train_test_split(train)

test_data = [x[0] for x in test]
test_labels = [x[1] for x in test]

def predict(pred_numbers):
    results = []
    for pred_number in pred_numbers:
        prediction = {"type": "",
                      "date": "",
                      "number": pred_number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

te = TypeExtractor()
te.train(train_data)
ne = NumberExtractor()
predicted = ne.predict(test_data, te.predict(test_data))
predicted = predict(predicted)
quality(predicted, test_labels)

{'date_accuracy': 0.0,
 'number_accuracy': 0.8276553106,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}